In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib nbagg
import os.path as op
import time

from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.optimizers import Adam
from keras_tqdm import TQDMNotebookCallback
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm_notebook

from fastmri_recon.data.test_generators import RandomShapeGenerator
from fastmri_recon.helpers.adversarial_training import compile_models, adversarial_training_loop
from fastmri_recon.helpers.image_tboard_cback import TensorBoardImage
from fastmri_recon.helpers.keras_utils import wasserstein_loss
from fastmri_recon.models.discriminator import discriminator_model, generator_containing_discriminator_multiple_outputs
from fastmri_recon.models.unet import unet

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
run_params = {
    'n_layers': 4, 
    'pool': 'max', 
    "layers_n_channels": [16, 32, 64, 128], 
    'layers_n_non_lins': 2,
}

im_size = 64

def generator_model():
    model = unet(input_size=(im_size, im_size, 1), **run_params, compile=False)
    model.name = 'Reconstructor'
    return model

In [3]:
# model definitions
g = generator_model()
# prev_run_id = 'unet_wo_lastrelu_af4_1572013433'
# prev_epoch = 500
# chkpt_path = f'checkpoints/{prev_run_id}-{prev_epoch}.hdf5'
# g.load_weights(chkpt_path)
d = discriminator_model(im_size)
d_on_g = generator_containing_discriminator_multiple_outputs(g, d, im_size=im_size)

W1031 11:48:30.469462 139862898484992 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:4070: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



In [4]:
# model compiling
perceptual_loss = 'mae'

discriminator_lr = 1e-4
d_on_reconstructor_lr = 1e-3

compile_models(
    d, 
    g, 
    d_on_g, 
    d_lr=discriminator_lr, 
    d_on_g_lr=d_on_reconstructor_lr, 
    perceptual_loss=perceptual_loss, 
    perceptual_weight=1.0,
)

In [5]:
n_batches_train = 1000
n_batches_val = 1
AF = 2
train_gen = RandomShapeGenerator(af=AF, n_shapes=100, size=im_size, batch_size=1).flow_z_filled_random_shapes()
val_gen = RandomShapeGenerator(af=AF, n_shapes=100, size=im_size, batch_size=1).flow_z_filled_random_shapes()

epoch_num = 100
critic_updates = 5

run_id = f'unet_gan_af{AF}_{int(time.time())}'
chkpt_path = f'checkpoints/{run_id}' + '-{epoch:02d}.hdf5'
print(run_id)

unet_gan_af2_1572518911


In [6]:
log_dir = op.join('logs', run_id)
tqdm_cb = TQDMNotebookCallback(metric_format="{name}: {value:e}")
tboard_cback = TensorBoard(
    log_dir=log_dir, 
    write_graph=True, 
    update_freq=50,
)
chkpt_cback = ModelCheckpoint(chkpt_path, period=epoch_num)
selected_slice = 0
data = next(val_gen)
tboard_image_cback = TensorBoardImage(
    log_dir=log_dir + '/images',
    image=data[1][selected_slice:selected_slice+1],
    # NOTE: for cross-domain slice has to be on kspace and mask
    model_input=data[0][selected_slice:selected_slice+1],
)

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/test_generators.py:33: ComplexWarning: Casting complex values to real discards the imaginary part
  kspaces[i, ..., 0] = kspace
/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/helpers/reconstruction.py:40: ComplexWarning: Casting complex values to real discards the imaginary part
  x_final[i] = fourier_op.adj_op(kspace)


In [7]:
# %%time
# adversarial_training_loop(
#     g, 
#     d, 
#     d_on_g, 
#     train_gen, 
#     val_gen=val_gen,
#     validation_steps=1,
#     n_epochs=epoch_num, 
#     n_batches=n_volumes_train, 
#     n_critic_updates=critic_updates,
#     callbacks=[tqdm_cb, tboard_cback, chkpt_cback, tboard_image_cback],
#     workers=35,
#     use_multiprocessing=True,
#     max_queue_size=35,
#     include_d_metrics=True,
# )

In [ ]:
%%time
#overfitting trial
adversarial_training_loop(
    g, 
    d, 
    d_on_g, 
    train_gen, 
#     val_gen=val_gen,
#     validation_steps=1,
    n_epochs=500, 
    n_batches=1, 
    n_critic_updates=5,
    callbacks=[tqdm_cb, tboard_cback, tboard_image_cback],
#     workers=35,
#     use_multiprocessing=True,
#     max_queue_size=35,
    include_d_metrics=True,
)

W1031 11:48:33.813407 139862898484992 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/callbacks/tensorboard_v1.py:200: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W1031 11:48:33.815080 139862898484992 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/callbacks/tensorboard_v1.py:203: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/skimage/util/dtype.py:135: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
W1031 11:48:34.035674 139862898484992 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/helpers/image_tboard_cback.py:26: The name tf.Summary is deprecated. Please use tf.compat.v1.Summary instead.



W1031 11:48:34.092073 139862898484992 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/skimage/util/dtype.py:135: UserWarning: Possible precision loss when converting from float32 to uint8
  .format(dtypeobj_in, dtypeobj_out))


/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/test_generators.py:33: ComplexWarning: Casting complex values to real discards the imaginary part
  kspaces[i, ..., 0] = kspace
/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/helpers/reconstruction.py:40: ComplexWarning: Casting complex values to real discards the imaginary part
  x_final[i] = fourier_op.adj_op(kspace)
